# A prototype implementation of experimental economics, part 1

by Gabriele Camera, Alexander Kurz, David Rojo Arjona

**Introduction:** This program presents a prototype of a software that can be used to run experiments in behavioural economics. Ultimately, there will be a client-server architecture with the server running the "economics engine" implemented below and participants of the experiment (subjects) providing decisions (actions) and extensions of the setting described will be investigated. Here only a simple economics engine is implemented. Subjects are simulated by functions that choose random actions. The cell at the end of this notebook contains code that runs a simulated experiment.

**To run this notebook** one can follow items 1 and 2 at [JuliaTutorials](https://github.com/JuliaAcademy/JuliaTutorials/blob/main/README.md).

**Background:** At each point in time, ***agents*** are matched pairwise, one being the ***producer***, the other the ***consumer***. Each producer chooses an ***action*** which can be thought of as providing ***consumption*** goods to the other agents in the group. Agents recieve ***utility*** (aka ***payoff***) from consumption.

**Naming conventions in the program:** "define" refers to parameters that the experimenter will adjust, "make" refers to data generated by the software, "choose" refers to input that will come from the participants of the experiement

Parameters that remain fixed throughout the dynamic evolution of the economy are:

In [1]:
const N = 6                        # define number of agents, must be even ≥ 2
const abar = 100                    # define upper bound for actions which will be in the interval [0,abar]
const δ = 0.9                       # define discounting factor (for future at even times, currently not needed)
;

Time is modelled by the non-negative integers. Currently time is not represented explicitly in the state. The global variables that describe the state of the economy at any given time are:

In [2]:
match = [0 for n=1:N]                      # match[j] is the matching partner of j

producer = [2 for n = 1:N]             # 0 for false, 1 for true, 2 for undefined
is_producer(j) = producer[j]==1 ? true : false
is_consumer(j) = producer[j]==0 ? true : false

consumption = [0 for n=1:N]            # consumption profile: agent j consumes consumption[j] at time t
    
action = [0 for n = 1:N]               # action profile: agent j produces action[j] (=0 if j is consumer)
;    

We also collect histories of actions:

In [3]:
history = []                           # history of all action profiles
;

We randomly pair agents in 'matching pairs':

In [4]:
# match agents randomly
function make_match() 
    listOfAgents = [n for n=1:N]
    for i in 1:N/2
        j=popfirst!(listOfAgents)            # take first agent from list
        index = rand(1:length(listOfAgents)) # choose random index in the remaining list
        k = listOfAgents[index]              # take agent at that index
        deleteat!(listOfAgents,index)        # remove agent from list
        global match[j]=k; match[k]=j        # update match
    end
end
;

In [5]:
# test make_match
make_match()
print(match)

# verify properties of match
for j in 1:N
    @assert(j == match[match[j]])  # pairwise matching
    @assert(j!=match[j])           # not matched to oneself
end

[3, 5, 1, 6, 2, 4]

Flip of a fair coin to determine who in each matching pair is the producer and who is the consumer. 

In [6]:
# flip a coin to determine which of the two agents in a matching pair is the producer/consumer
function make_producer()
    global producer = [2 for n = 1:N] # erase previous definition
    for i in 1:N
        if producer[i] == 2
            coinflip = rand(0:1)
            if coinflip == 1 
                producer[i] = 1; producer[match[i]] = 0 # if i produces match[i] does not
            else
                producer[i] = 0; producer[match[i]] = 1 # if i does not produce match[i] does
            end
        end
    end
end
;

In [7]:
# test make producer 
make_producer(); println(producer)

for i in 1:N
    @assert(producer[i]!=producer[match[i]]) # must hold for producers at odd times
end

[1, 1, 0, 1, 0, 0]


At odd times, producers choose and action in $[0,\bar a]$. 

In [8]:
# agents choose actions (if they are producers)
function choose_action()
    global action = [is_producer(j) ? rand(0:abar) : 0 for j=1:N]              # production
end
;

In [9]:
# test choice of actions

make_producer(); print("producer: "); println(producer); 
choose_action(); print("action:   "); println(action); 

for i in 1:N
    @assert(!(producer[i]==0) || action[i] == 0) # if consumer then action=0
end

producer: [1, 1, 0, 0, 0, 1]
action:   [41, 96, 0, 0, 0, 17]


If `j` is a consumer than `consumption[j]` equals the production `action[match[j]]` produced by the match of `j`.

In [10]:
function make_consumption()
    global consumption = [is_consumer(j) ? action[match[j]] : 0 for j=1:N]
end
;

Each consumer receives utility from consuming $c$ goods; payoffs are given by the utility from consumption minus the costs of the own production:

In [11]:
function utility(c) 
    c                                      # define utility function here
end

function make_payoff()
    global payoff = [utility(consumption[j]-action[j]) for j=1:N]
end
;

In [12]:
# test 

make_producer(); print("producer: "); println(producer); 
choose_action(); print("action:   "); println(action); 
make_consumption(); print("consumption:  "); println(consumption)
make_payoff(); print("payoff:  "); println(payoff)

# check that what one partner consumes is exactly what the other partner produces
for j in 1:N 
    @assert(is_producer(j) ? action[j]==consumption[match[j]] : consumption[j]==action[match[j]])
end

# check that total sum of payoffs is zero
@assert(sum(payoff)==0)

producer: [1, 0, 0, 0, 1, 1]
action:   [61, 0, 0, 0, 95, 71]
consumption:  [0, 95, 61, 71, 0, 0]
payoff:  [-61, 95, 61, 71, -95, -71]


In [13]:
# test 
history = [] # clear history
for t in 1:4 # can we start at 0?
    print("t=");println(t)
    make_match(); print("match: "); println(match)                              # make matchings
    make_producer(); print("producer: "); println(producer)                     # flip coins to determine producers
    choose_action(); print("action: "); println(action)                         # choose action for each producer
    make_consumption(); print("consumption: "); println(consumption)            # compute consumptions 
    make_payoff(); print("payoff: "); println(payoff)                           # compute payoffs 
    println()
    push!(history,action); println(history); println()
end
;

t=1
match: [3, 4, 1, 2, 6, 5]
producer: [1, 1, 0, 0, 1, 0]
action: [17, 64, 0, 0, 66, 0]
consumption: [0, 0, 17, 64, 0, 66]
payoff: [-17, -64, 17, 64, -66, 66]

Any[[17, 64, 0, 0, 66, 0]]

t=2
match: [6, 3, 2, 5, 4, 1]
producer: [0, 0, 1, 1, 0, 1]
action: [0, 0, 5, 100, 0, 75]
consumption: [75, 5, 0, 0, 100, 0]
payoff: [75, 5, -5, -100, 100, -75]

Any[[17, 64, 0, 0, 66, 0], [0, 0, 5, 100, 0, 75]]

t=3
match: [6, 5, 4, 3, 2, 1]
producer: [1, 0, 1, 0, 1, 0]
action: [74, 0, 10, 0, 4, 0]
consumption: [0, 4, 0, 10, 0, 74]
payoff: [-74, 4, -10, 10, -4, 74]

Any[[17, 64, 0, 0, 66, 0], [0, 0, 5, 100, 0, 75], [74, 0, 10, 0, 4, 0]]

t=4
match: [6, 3, 2, 5, 4, 1]
producer: [1, 1, 0, 1, 0, 0]
action: [52, 60, 0, 32, 0, 0]
consumption: [0, 0, 60, 0, 32, 52]
payoff: [-52, -60, 60, -32, 32, 52]

Any[[17, 64, 0, 0, 66, 0], [0, 0, 5, 100, 0, 75], [74, 0, 10, 0, 4, 0], [52, 60, 0, 32, 0, 0]]



**Remark:** Updating a global variable such as `history` in Julia can be subtle. I wrote a post on this at [stackoverflow](https://stackoverflow.com/questions/75052389/why-does-updating-a-global-vector-with-for-loop-and-with-list-comprehension-in-j).